In [1]:
:opt no-lint

# Untyped $𝜆$-calculus
타입없는 람다계산법의 문법과 줄일 수 있는 식(reducible expression)의 개념 등에 대해 알아본다.

"줄일 수 있는 식"이라는 말이 너무 길기 때문에 '주는식'(redex)과 같은 줄임말을 보통 더 많이 사용한다.

## 문법
$  x \in \textit{Nm} $

$  e \in \textit{Tm}
\\ e ::=~ x ~\mid~ (\lambda x.e) ~\mid~ (e_1~e_2)
$

In [2]:
-- 변수 이름은 문자열 나타낸다
type Nm = String

-- 람다식 문법 구조
data Tm = Var Nm       -- x
        | Lam Nm Tm    -- (λx.e)
        | App Tm Tm    -- (e1 e2)
        deriving (Show, Eq)

In [3]:
-- 람다식을 보기좋게 문자열로 변환해주는 함수
ppTm (Var x) = x
ppTm (Lam x e) = "\\" ++ x ++ " -> " ++ ppTm e
ppTm (App e1 e2) = pp1 e1 ++ " " ++ pp2 e2
  where
  pp1 e@(Lam{}) = paren (ppTm e)
  pp1 e         = ppTm e
  pp2 e@(Var{}) = ppTm e
  pp2 e         = paren (ppTm e)

paren s = "(" ++ s ++ ")"
brack s = "[" ++ s ++ "]"
latex s = "$" ++ s ++ "$"

-- 람다식을 보기좋게 TeX 코드로 변환해주는 함수
texTm (Var x) = x
texTm (Lam x e) = "\\lambda " ++ x ++ "." ++ texTm e
texTm (App e1 e2) = tex1 e1 ++ "~" ++ tex2 e2
  where
  tex1 e@(Lam{}) = paren (texTm e)
  tex1 t         = texTm t
  tex2 s@(Var{}) = texTm s
  tex2 s         = paren (texTm s)

In [4]:
idTm = Lam "x" (Var "x")
ttTm = Lam "x" (Lam "y" (Var "x")) 
ffTm = Lam "x" (Lam "y" (Var "y")) 

In [5]:
putStr $ ppTm idTm
putStr $ ppTm ttTm
putStr $ ppTm ffTm

\x -> x

\x -> \y -> x

\x -> \y -> y

In [6]:
import IHaskell.Display

html . latex $ texTm idTm
html . latex $ texTm ttTm
html . latex $ texTm ffTm
html . latex $ texTm (App (App (Var "x") (Var "y")) (Var "z"))
html . latex $ texTm (App (Var "x") (App (Var "y") (Var "z")))
html . latex $ texTm (App (App ffTm idTm) ttTm)
html . latex $ texTm (App ffTm (App idTm ttTm))

$\lambda x.x$

$\lambda x.\lambda y.x$

$\lambda x.\lambda y.y$

$x~y~z$

$x~(y~z)$

$(\lambda x.\lambda y.y)~(\lambda x.x)~(\lambda x.\lambda y.x)$

$(\lambda x.\lambda y.y)~((\lambda x.x)~(\lambda x.\lambda y.x))$

## 부분식
람다식의 부분식은 그 식의 일부로 나타나는 람다식이다. 전체 식 자체도 자기 자신의 부분식이라고 하자.

하스켈 데이타 타입 `Tm`으로 표현된 어떤 람다식의 부분식을 모두 나열하는 리스트를 구하는 함수를 다음과 같이 작성할 수 있다.

In [27]:
subTm :: Tm -> [Tm]
subTm (Var x) = [(Var x)]
subTm (Lam x e1) = (Lam x e1) : subTm e1
subTm (App e1 e2) = (App e1 e2) : subTm e1 ++ subTm e2

In [28]:
e = App (Lam "x" (Var "x")) (Lam "y" (Var "y"))

subTm e

map (html . latex . texTm) (subTm e)

[App (Lam "x" (Var "x")) (Lam "y" (Var "y")),Lam "x" (Var "x"),Var "x",Lam "y" (Var "y"),Var "y"]

$(\lambda x.x)~(\lambda y.y)$

$\lambda x.x$

$x$

$\lambda y.y$

$y$

패턴매칭에서 부분별로 이름을 붙임과 동시에 구조 전체에도 이름을 붙여주려면 `@`을 활용한다.
예컨대 위의 `subTm`과 같은 함수를 다음과 `@`패턴을 활용하면 더 간결하게 정의되며,
컴파일러가 구조 전체를 함수 결과에서 그대로 참조할 수 있는 추가 정보가 돌 수 있으므로
컴파일된 코드의 성능이 개선될 가능성도 있다.

In [29]:
subTm' :: Tm -> [Tm]
subTm' e@(Var _) = [e]
subTm' e@(Lam x e1) = e : subTm e1
subTm' e@(App e1 e2) = e : subTm e1 ++ subTm e2

In [30]:
e = App (Lam "x" (Var "x")) (Lam "y" (Var "y"))

subTm' e

map (html . latex . texTm) (subTm' e)

[App (Lam "x" (Var "x")) (Lam "y" (Var "y")),Lam "x" (Var "x"),Var "x",Lam "y" (Var "y"),Var "y"]

$(\lambda x.x)~(\lambda y.y)$

$\lambda x.x$

$x$

$\lambda y.y$

$y$

## 주는식(redex)
람다계산법의 실행의미는 그 문법만큼이나 매우 단순하다.
함수정의식 $(\lambda x.e)$를 인자 $e_2$에 적용한
함수적용식 $((\lambda x.e)~e_2)$는 곧바로 줄일 수 있는
주는식(redex)이며 그것을 한 걸음 줄인 결과는
$e$에 나타나는 $x$를 $e_2$로 치환한 식,
즉 $x$의 자리에 $x$대신 $e_2$로 끼워넣은 식이 된다.
이러한 계산 규칙을 $\beta$-줄이기($\beta$-reduction)이라고 하며
아래와 같이 나타낸다.

$((\lambda x. e)~e_2) \longrightarrow \{x\mapsto e_2\}e$

여기에서 $\{x\mapsto e_2\}e$는 바로 $e$에서 $x$를 $e_2$로 치환한 식을 의미하는 표기법이다.
참고로 위의 $\beta$-줄이기 규칙을 핵심으로 하되, 어떤 식이 그 자체로는 $\beta$-줄이기 규칙의
적용 대상이 아니더라도 규칙이 적용되는 부분을 찾아 그 부분의 식에 $\beta$-줄이기를 적용하는
규칙들을 추가로 정의할 수 있다. 예컨대, $\big(\big((\lambda x.x)~(\lambda y.y)\big)~(\lambda z.z)\big)$는
그 자체로는 식 전체에 $\beta$-줄이기 규칙을 적용할 수 없지만
그 부분식인 $\big((\lambda x.x)~(\lambda y.y)\big)$에는 $\beta$-줄이기 규칙을 적용할 수 있다.
사실 부분식에 $\beta$-줄이기를 어떻게 적용할지는 여러 가지
방법 혹은 전략이 가능한데 지금은 일단 적당히 아무 곳에나 적용한다고 하고 넘어가기로 하고 이후에
다시 이에 대해 생각해 보겠다.

하스켈 데이타 타입 `Tm`으로 표현된 어떤 람다식이
그 자체로 $\beta$-줄이기 규칙을 바로 적용할 수 있는 주는식(redex)인지
검사하는 함수를 다음과 같이 작성할 수 있다.

In [31]:
isRedEx :: Tm -> Bool
isRedEx (App (Lam{}) _) = True
isRedEx _               = False

In [32]:
e1 = idTm
e2 = App idTm idTm
e3 = (Lam "x" e2)

html . latex $ texTm e1
isRedEx e1
html . latex $ texTm e2
isRedEx e2
html . latex $ texTm e3
isRedEx e3

$\lambda x.x$

False

$(\lambda x.x)~(\lambda x.x)$

True

$\lambda x.(\lambda x.x)~(\lambda x.x)$

False

주어진 식 자체로는 주는식(redex)이 아니더라도 어떤 부분의 부분식에 주는식을 포함하고 있을 수도 있다.
그 자체로 주는 식인 경우 뿐만 아니라 부분식으로 주는식을 포함하는 식인지까지 검사하는 `hasRedEx` 함수를
다음과 같이 작성할 수 있다.

In [33]:
hasRedEx :: Tm -> Bool
hasRedEx (Var _) = False
hasRedEx (Lam _ e1) = hasRedEx e1
hasRedEx e@(App e1 e2) = isRedEx e || hasRedEx e1 || hasRedEx e2

In [34]:
e1 = idTm
e2 = App idTm idTm
e3 = (Lam "x" e2)

html . latex $ texTm e1
hasRedEx e1
html . latex $ texTm e2
hasRedEx e2
html . latex $ texTm e3
hasRedEx e3

$\lambda x.x$

False

$(\lambda x.x)~(\lambda x.x)$

True

$\lambda x.(\lambda x.x)~(\lambda x.x)$

True

참고로 위와 같은 일을 하는 다른 방법으로도 정의할 수 있다.
앞서 정의한 `subTm` 함수로 부분식을 모두 나열하여
그 중 하나라도 주는 식이 있는지 `isRedEx` 함수와 함께
표준라이브러리 고차함수 `any`를 활용하면 알아볼 수 있으므로
다음과 같은 정의도 가능하다.

In [36]:
hasRedEx' = any isRedEx . subTm

In [37]:
e1 = idTm
e2 = App idTm idTm
e3 = (Lam "x" e2)

html . latex $ texTm e1
hasRedEx' e1
html . latex $ texTm e2
hasRedEx' e2
html . latex $ texTm e3
hasRedEx' e3

$\lambda x.x$

False

$(\lambda x.x)~(\lambda x.x)$

True

$\lambda x.(\lambda x.x)~(\lambda x.x)$

True

----
### HW2-pl2019 첫번째 문제 (2점 + 보너스 1점)
식에 포함된 주는식(redex)의 개수를 세는 함수 `countRedEx :: Tm -> Int`의 정의를 완성하고
이 함수를 활용하는 테스트를 아래 주어진 테스트 외에 자신이 직접 3개 이상 작성하라.
단, 주어진 테스트를 포함하여 테스트의 결과값은 서로 모두 달라야 한다.
함수를 맞게 작성했더라도 이 조건을 만족하는 테스트를 작성하지 않으면 0점이다.

보너스 점수를 얻으려면 또 다른 방법으로 같은 일을 하는 함수 `countRedEx' :: Tm -> Int`를 작성해 보고,
앞서 작성한 같은 테스트를 새로 다른 방법으로 작성한 함수로도 실행해 보라.

In [15]:
-- countRedEx :: Tm -> Int

In [16]:
-- 테스트 0
e1 =  Lam "f" (App idTm (App idTm (Var "f" `App` Var "f"))) `App` App idTm idTm
html . latex $ texTm e1
countRedEx e1 -- 제대로 완성하면 결과가 4가 나올 것이다

$(\lambda f.(\lambda x.x)~((\lambda x.x)~(f~f)))~((\lambda x.x)~(\lambda x.x))$

: 

In [17]:
-- 테스트 1

In [18]:
-- 테스트 2

In [19]:
-- 테스트 3

----

In [20]:
-- texTm을 적절히 변경하여 함수적용식 (e1 e2)에서
-- 함수 부분인 e1에 빨강 인자 부분인 e2에 파랑 사각형 테두리 추가
texTmColor (Var x) = x
texTmColor (Lam x e) = "\\lambda " ++ x ++ "." ++ texTmColor e
texTmColor (App e1 e2) = box1(tex1 e1) ++ "~" ++ box2(tex2 e2)
  where
  tex1 e@(Lam{}) = paren (texTmColor e)
  tex1 t         = texTmColor t
  tex2 s@(Var{}) = texTmColor s
  tex2 s         = paren (texTmColor s)
  box1 = red  . boxed . black
  box2 = blue . boxed . black

color c s = "\\color{"++c++"}{"++s++"}"
boxed s = "\\boxed{"++s++"}"

black = color "black"
red = color "red"
blue = color "blue"

In [21]:
e1 = Lam "f" (App idTm (App idTm (Var "f" `App` Var "f"))) `App` App idTm idTm
putStr $ texTmColor e1
html . latex $ texTmColor e1

\color{red}{\boxed{\color{black}{(\lambda f.\color{red}{\boxed{\color{black}{(\lambda x.x)}}}~\color{blue}{\boxed{\color{black}{(\color{red}{\boxed{\color{black}{(\lambda x.x)}}}~\color{blue}{\boxed{\color{black}{(\color{red}{\boxed{\color{black}{f}}}~\color{blue}{\boxed{\color{black}{f}}})}}})}}})}}}~\color{blue}{\boxed{\color{black}{(\color{red}{\boxed{\color{black}{(\lambda x.x)}}}~\color{blue}{\boxed{\color{black}{(\lambda x.x)}}})}}}

$\color{red}{\boxed{\color{black}{(\lambda f.\color{red}{\boxed{\color{black}{(\lambda x.x)}}}~\color{blue}{\boxed{\color{black}{(\color{red}{\boxed{\color{black}{(\lambda x.x)}}}~\color{blue}{\boxed{\color{black}{(\color{red}{\boxed{\color{black}{f}}}~\color{blue}{\boxed{\color{black}{f}}})}}})}}})}}}~\color{blue}{\boxed{\color{black}{(\color{red}{\boxed{\color{black}{(\lambda x.x)}}}~\color{blue}{\boxed{\color{black}{(\lambda x.x)}}})}}}$

----
### HW2-pl2019 두번째 문제 (3점)
`texTmColor`처럼 모든 함수적용식 $(e_1~e_2)$에 빨강/파랑 사각형 테두리를 표시하는 것이 아니라
주는식(redex)인 경우에만, 즉 $((\lambda x.e)~e_2)$ 형태인 경우에만,
빨강/파랑 사각형 테두리로 표시하는 `texTmRedEx` 함수를 정의하라.
(힌트: 일단 `texTmColor`의 정의를 복사/붙여넣기 해서 `texTmColor`를 모두 `texTmRedEx`로 바꿔주고 난 다음 어느 부분을 고쳐야 할지 잘 생각해 보라.)

예컨대, 앞서 본 같은 예제를 `texTmRedEx`으로 실행한 결과는 다음과 같아야 한다.
```haskell
e1 = Lam "f" (App idTm (App idTm (Var "f" `App` Var "f"))) `App` App idTm idTm
putStr(texTmRedEx e1)
html(latex(texTmRedEx e1))
```
`\color{red}{\boxed{\color{black}{(\lambda f.\color{red}{\boxed{\color{black}{(\lambda x.x)}}}~\color{blue}{\boxed{\color{black}{(\color{red}{\boxed{\color{black}{(\lambda x.x)}}}~\color{blue}{\boxed{\color{black}{(f~f)}}})}}})}}}~\color{blue}{\boxed{\color{black}{(\color{red}{\boxed{\color{black}{(\lambda x.x)}}}~\color{blue}{\boxed{\color{black}{(\lambda x.x)}}})}}}`

$\color{red}{\boxed{\color{black}{(\lambda f.\color{red}{\boxed{\color{black}{(\lambda x.x)}}}~\color{blue}{\boxed{\color{black}{(\color{red}{\boxed{\color{black}{(\lambda x.x)}}}~\color{blue}{\boxed{\color{black}{(f~f)}}})}}})}}}~\color{blue}{\boxed{\color{black}{(\color{red}{\boxed{\color{black}{(\lambda x.x)}}}~\color{blue}{\boxed{\color{black}{(\lambda x.x)}}})}}}$



In [22]:
-- texTmRedEx :: Tm -> String

In [23]:
-- 테스트 0
{-
e1 = Lam "f" (App idTm (App idTm (Var "f" `App` Var "f"))) `App` App idTm idTm
putStr $ texTmRedEx e1
html. latex $ texTmRedEx e1
-}

In [24]:
-- 테스트 1

In [25]:
-- 테스트 2

In [26]:
-- 테스트 3

----

## 표준형(normal form)

표준형은 규칙에 따라 더 이상 줄일 수 있는 부분식을 포함하지 않는 형태의 람다식을 말한다.
구체적으로 $\beta$-줄이기로 더 이상 줄일 수 있는 부분식이 전혀 포함하지 않는 람다식을 $\beta$-표준형이라 일컫는다.

주어진 람다식 $e$에 $\beta$-줄이기를 반복적으로 적용해 얻을 수 있는 $\beta$-표준형을
$e$의 $\beta$-표준형이라 말하는데, 타입없는 람다계산법에서는
주어진 람다식의 $\beta$-표준형이 존재한다면 유일하다는
**처치-로서 정리(Church-Rosser theorem)**가 성립힌다.

람다계산법의 처치-로서 정리를 직접 설명하기보다는
우선 직관적으로 처치-로서 정리가 어떤 성질을 의미하는지 덧셈식을 통해 간단히 개념만 알아보자.
 
예를 들어 (1 + 2) + (4 + 5) 라는 덧셈식이 있다면 어떤 덧셈을 먼저 하든 관계없이 항상 같은 결과값을 얻는다.
즉 덧셈식에서는 덧셈을 줄여나가는 것에 대한 처치-로서 정리의 성질이 성립한다고 말할 수 있다.

$(1 + 2) + (4 + 5) ~\longrightarrow~ 3 + (4 + 5) ~\longrightarrow~ 3 + 9 ~\longrightarrow~ 12$

$(1 + 2) + (4 + 5) ~\longrightarrow~ (1 + 2) + 9 ~\longrightarrow~ 3 + 9 ~\longrightarrow~ 12$

하지만 타입없는 람다계산법의 경우 계산이 일반적으로 항상 끝난다는 보장이 없다.
즉 어떤 람다식은 표준형이 아예 존재하지 않을 수도 있으며
표준형이 존재하더라도 어떤 줄일식을 먼저 계산하는가에 따라
계산이 끝날 수도 있고 끝나지 않을 수도 있다. 그래서 타입없는
람다계산법에서의 처치-로서 정리는 표준형이 존재하는 식의 경우
표준형에 도달할 수 있는 모든 계산 경로 끝에 있는 표준형이 모두 "같다"는 의미이다.

## $\alpha$-변환 ($\alpha$-conversion)

잠깐, 근데 "같다"는 게 무엇인지 우리가 아직 정확히 정의하지 않았다.

TODO